# Run experiment multi-output regression


---


The aim of this script is to run the experiment in order to train a multioutput regression neural network to predict the size of cell parameters of different crystal structures. Users have to load the crystal structure dataset, read it, select which are the features to be used to train the model, as well as, which are the response variables. 

After that, dataset will be splitted in 80% for training and 20% for test dataset. Training dataset will be splitted again in 80% for training and 20% for validation, during the hyperparameters optimization. After that, the best hyperparameters will be used to fit agai the neural network on the full training dataset.

## Import library

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import os

# Data Manipulation
import numpy as np
import pandas as pd
from scipy.stats import reciprocal

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# TensorFlow / Keras 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsolutePercentageError
import keras_tuner as kt
import tensorflow as tf

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import model_selection

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

## Load the Data


---


User must enter the name of the path in which dataset is stored. After that, we will check if the directory exists and if it is empty or not.

### *Check directory and files*

In [ ]:
def read_dataset ():
  dataset = ''
  path_name = input('Enter the path name for dataset: ')
  path_name = '/content/' + path_name

  if not os.path.exists(path_name):
      print('Error! Invalid path selected.')
  else:
      print(path_name + ' is a valid path.')

      if not os.listdir(path_name):
        print("Warning! Empty directory.")
      else:
        file_name = input('Enter the file name for dataset: ')
        dataset = pd.read_csv(path_name + '/' + file_name + '.csv', sep = ';', index_col = 'ID_Observations' )
  return dataset

### *Load the dataset*

In [ ]:
y_coord_dataset = read_dataset()

In [ ]:
print('y_coord_dataset shape: {}'.format(y_coord_dataset.shape))
print('\n data types: \n{}'.format(y_coord_dataset.dtypes))
print('\ny_coord_dataset content: \n')
y_coord_dataset

## Insert details on dataset features


---


Users have to specify which variables will be used as features and which will be response features.

In [ ]:
def insert_dataset_structure_details(final_dataset):
    print('============================================================================')
    print('Insert dataset structure details');

    while 1:
        print('----------------------------------------------------------------------------')
        use_volume = input('\n1) Do you want to use volume as a feature in the experiment? [Y|N]: ');
        if use_volume.lower() == 'y':
          use_volume = True;
          break;
        elif use_volume.lower() == 'n':
          use_volume = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        use_total_n_peaks = input('\n2) Do you want to use total_n_peaks as a feature in the experiment? [Y|N]: ');
        if use_total_n_peaks.lower() == 'y':
          use_total_n_peaks = True;
          break;
        elif use_total_n_peaks.lower() == 'n':
          use_total_n_peaks = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        use_max_peaks = input('\n3) Do you want to use max_peaks as a feature in the experiment? [Y|N]: ');
        if use_max_peaks.lower() == 'y':
          use_max_peaks = True;
          break;
        elif use_max_peaks.lower() == 'n':
          use_max_peaks = False;
          break;
   
    while 1:
        print('----------------------------------------------------------------------------')
        a_is_response = input('\n4) "a" is a response feature ? [Y|N]: ');
        if a_is_response.lower() == 'y':
          a_is_response = True;
          break;
        elif a_is_response.lower() == 'n':
          a_is_response = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        b_is_response = input('\n5) "b" is a response feature ? [Y|N]: ');
        if b_is_response.lower() == 'y':
          b_is_response = True;
          break;
        elif b_is_response.lower() == 'n':
          b_is_response = False;
          break;

    while 1:
        print('----------------------------------------------------------------------------')
        c_is_response = input('\n6) "c" is a response feature ? [Y|N]: ');
        if c_is_response.lower() == 'y':
          c_is_response = True;
          break;
        elif c_is_response.lower() == 'n':
          c_is_response = False;
          break;

    print('============================================================================')

    response_features_name = []

    if not use_total_n_peaks:
      final_dataset.drop('Total_n_peaks', axis=1, inplace=True)
    
    if not use_max_peaks:
      final_dataset.drop('Max_peaks_position', axis=1, inplace=True)

    if not use_volume:
      final_dataset.drop('Volume', axis=1, inplace=True)
    
    if not a_is_response:
      final_dataset.drop('a', axis=1, inplace=True)
    else:
      response_features_name.append('a')
    
    if not b_is_response:
      final_dataset.drop('b', axis=1, inplace=True)
    else:
      response_features_name.append('b')
    
    if not c_is_response:
      final_dataset.drop('c', axis=1, inplace=True)  
    else:
      response_features_name.append('c')

    final_dataset.drop('Crystal_Structure_Type', axis=1, inplace=True)

    return final_dataset, response_features_name

In [ ]:
final_dataset_for_experiment, response_features_name = insert_dataset_structure_details(y_coord_dataset.copy())
print('The response features for this crystal structure are: {}'.format(response_features_name))
print('The final dataset with features is:\n')
final_dataset_for_experiment

## Splitting data for regression task

### *Separating Input Features and Output Features*

In [ ]:
x = final_dataset_for_experiment[final_dataset_for_experiment.columns.difference(response_features_name)]
print('The dataset has the following number of input features: {:d}'.format(x.shape[1]))
print('Input features to be used for training are: ')
x

In [ ]:
y = final_dataset_for_experiment[response_features_name]
print('The dataset has the following number of output features: {:d}'.format(y.shape[1]))
print('Output features are:')
y

### *Splitting the dataset into training set, validation set and test set*
We use 80% of the original dataset as training dataset and the remaining 20% to test the model.
The training set (80%) is splitted again in training dataset (80%) and validation dataset (20%).

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, shuffle= True )

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, shuffle= True)

In [ ]:
print('Training-Validation-Test dataset shape')
print('------------------------------------------------------------------------')
print('- The training set has the following number of observations: {:d}'.format(y_train.shape[0])) 
print('- The validation set has the following number of observations: {:d}'.format(y_val.shape[0]))
print('- The test set has the following number of observations: {:d}'.format(y_test.shape[0]))

#### *Boxplot distribution of response in training, validation and test set*

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3)
y_train.boxplot(column=y_train.columns.tolist(), ax=axes[0])
axes[0].set_title('Training')

y_val.boxplot(column=y_val.columns.tolist(), ax=axes[1])
axes[1].set_title('Validation')

y_test.boxplot(column=y_test.columns.tolist(), ax=axes[2])
axes[2].set_title('Test')

fig.tight_layout(h_pad=1)
fig.suptitle('Output distribution')
plt.subplots_adjust(top=0.85)


In [ ]:
plt.savefig('boxplot_cube_output_distribution.png')

## Training multi-output regression neural network model

---

After splitting the data into training, validation and testing set, it's time to train our neural network model. 

### *Define the neural network architectures and search space for hyperparameters*

In [ ]:
def build_model(hp):
  model = Sequential(name='Multi_Output_Neural_Network')

  model.add(Dense(units=hp.Int('units_input_layer', min_value=32, max_value=512, step=32),
                  input_shape = (x_train.shape[1],),
                  name='Input_Layer',
                  kernel_initializer='he_normal',
                  bias_initializer='zeros'
                  )
  )
                  
  model.add(Dropout(rate=hp.Float('dropout_rate_input_layer', 
                                     min_value=0.0, max_value=0.9, step=0.1), 
                    name=f'Dropout_Input_Layer'))
  
  # Optimize the number of hidden layers
  for i in range(hp.Int('num_hidden_layers', 1, 6)):
    model.add(Dense(units=hp.Int(f'units_hidden_layer_{i}', min_value=32, max_value=512, step=32),
                    activation=hp.Choice(f'activation_fun_hidden_layer_{i}', ['relu','tanh', 'softmax', 'softplus', 'selu', 'elu', 'softsign']), 
                    name=f'Hidden_Layer_{i+1}',
                    kernel_initializer='he_normal',
                    bias_initializer='zeros'))
    
    model.add(Dropout(rate=hp.Float(f'dropout_rate_hidden_layer_{i}', 
                                     min_value=0.0, max_value=0.9, step=0.1), name=f'Dropout_Layer_{i+1}'))
    
  # Add an output layer with n neurons, one for each output
  model.add(Dense(y_train.shape[1], activation='linear', name='Output_Layer'))

  # Define the optimizer learning rate as a hyperparameter.
  learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
  model.compile(
      optimizer=Adam(learning_rate=learning_rate),
      loss=['mean_squared_error'],
      metrics=[RootMeanSquaredError(), MeanAbsolutePercentageError()], 
      )
  return model

In [ ]:
build_model(kt.HyperParameters())

### *Define settings for hyperparameters search*


*   **MAX_TRIALS** - Number of hyperparameter combinations that will be tested by the tuner
*   **EXECUTION_PER_TRIAL** - The number of models that should be built and fit for each trial. Different trials have different hyperparameter values. The executions within the same trial have the same hyperparameter values. The purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. 
*  **MAX_EPOCHS** - Maximum number of epochs to train one model
*  **MINI_BATCH** - Size of subset of the dataset used to take another step in the learning process.Instead of waiting for the model to compute the whole dataset, we’re able to update its parameters more frequently. This reduces the risk of getting stuck at a local minimum, since different batches will be considered at each iteration, granting a robust convergence.
*  **PATIENCE** - Number of epochs with no improvement after which training will be stopped.



In [ ]:
MAX_TRIALS = 50
EXECUTION_PER_TRIAL = 3
MAX_EPOCHS = 250
MINI_BATCH_SIZE = 32
PATIENCE = 5

### *Start the search*

In [ ]:
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective=kt.Objective('val_loss', direction='min'),
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory='neural-network-optimization',
    project_name='cubic-optimization-neural-network-tot-peaks',
    overwrite=True,
    )

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x_train.to_numpy(),
             y_train.to_numpy(), 
             epochs=MAX_EPOCHS,
             batch_size=MINI_BATCH_SIZE,
             validation_data=(x_val.to_numpy(), y_val.to_numpy()),
             callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1, restore_best_weights = True),
                        keras.callbacks.TensorBoard('neural-network-optimization/cubic-optimization-neural-network-tot-peaks_logs')
                      ],
             )

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir neural-network-optimization/cubic-optimization-neural-network-tot-n-peaks_logs

In [ ]:
tuner.results_summary()

### *Retrain the model with the best hyperparameters*
After the hyperparameters tuning, we take the best hyperparameters to train the model using the all training dataset (*x_train + x_val*).

In [ ]:
x_train = pd.concat([x_train, x_val], axis=0)
y_train = pd.concat([y_train, y_val], axis=0)

In [ ]:
print('The hyperparameter search is complete.\nThe optimal hyperparameters are: \n')
tuner.get_best_hyperparameters()[0].values

In [ ]:
best_model = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
training_history = best_model.fit(x_train.to_numpy(), 
                                  y_train.to_numpy(), 
                                  epochs=MAX_EPOCHS,
                                  batch_size=MINI_BATCH_SIZE, 
                                  )

### *Show trained neural network architecture*


In [ ]:
best_model.summary()

In [ ]:
plot_model(best_model)

## Predict and evaluate the trained neural network on test set

### *Predict on test set*

In [ ]:
y_test_pred = best_model.predict(x_test.to_numpy())

### *Save predicted values*

In [118]:
y_features_name = y_test.columns.tolist()
features_col_name = {}

features_col_name = {y_features_name[i] + '_pred' for i in range(0, len(y_features_name))}

summary_prediction = y_test.join(pd.DataFrame(y_test_pred, index=y_test.index, columns=features_col_name))
summary_prediction = summary_prediction.reindex(sorted(summary_prediction.columns), axis=1)
summary_prediction.to_csv('summary_prediction_cube_test.csv', sep=';', index=True, header=True)
summary_prediction

,a,a_pred
ID_Observations,,
7231303,35.2860,34.558456
4335735,6.3555,7.623268
7107289,26.2896,26.076038
7105731,26.4013,26.599613
7038163,33.4530,33.572762
...,...,...
4329655,23.0340,23.220886
7225728,25.7703,26.928612
4513571,10.5186,11.846985


### *Evaluate the trained model on test set*

In [ ]:
r2_result=[]
corr_coeff_result=[]
for i in range(0,summary_prediction.shape[1],2):
  r2_result.append(r2_score(summary_prediction.iloc[i], summary_prediction.iloc[i+1]))
  corr_coeff_result.append(summary_prediction.iloc[:,[i,i+1]].corr().iloc[0,1])

[mse_test, rmse_test, mape_test] = best_model.evaluate(x_test.to_numpy(),y_test.to_numpy())
print('- r2_score for output {}: {}'.format(y_features_name,r2_result))
print('- corr_coeff for output {}: {}'.format(y_features_name,corr_coeff_result))

##### *Create PWB Table*

In [ ]:
def compute_pwbX(df, threshold, output_name):
  df_inrange = pd.DataFrame(index=df.index, columns=output_name)

  j=0
  for i in range(0, df.shape[1],2):
    obs = df.iloc[:,0]
    pred = df.iloc[:,1]
    min_bound=obs-(obs*threshold/100)
    max_bound=obs+(obs*threshold/100)
    df_inrange.iloc[:,j] = pred.between(min_bound,max_bound)
    j+=1

  return (df_inrange.all(axis='columns').values.sum())*100/df.shape[0]

In [ ]:
pwbX = [1, 5, 10, 20]
pwbXIndex = ['PWB' + str(pwbX[i]) for i in range(0, len(pwbX))]
pwbXIndex

pwbTable = pd.DataFrame(index=pwbXIndex, columns=['% samples'])
pwbTable

for i in range(0, len(pwbX)):
  pwbTable.iloc[i,0] = compute_pwbX(summary_prediction, pwbX[i],y_features_name)

pwbTable

### *Save trained model*

In [ ]:
best_model.save('cubic_neural_network.h5')

## Define function to plot multi-output regression results

### *Perfect fit plot*
*Perfect fit plot* will display a straight black line meaning real observations value are equals to predicted values, and a blue points which will represent the observations. We will plot real observations value again predicted values.

In [ ]:
def plot_perfect_fit(df, output_names, title_fig):

  n_output = df.shape[1]
  
  fig = make_subplots(
      rows=1, cols= round(n_output/2),
      subplot_titles=['<b>' + name_output + '</b>' for name_output in output_names],
  )

  max_lim_axis = round(max(df.max(axis=0))+1)

  min_lim_axis = round(min(df.min(axis=0)))
  if min_lim_axis > 0:
    min_lim_axis = 0
  
  fit_point = np.linspace(0, max_lim_axis, max_lim_axis)

  j = 1

  for i in range(0,n_output,2):
    obs = df.iloc[:, i].to_numpy()
    pred = df.iloc[:, i+1].to_numpy()

    fig.add_trace(go.Scatter(x=obs, y=pred, mode='markers', marker_color='#1F77B4', 
                             marker_size=8, name='Observations'), row=1, col=j)
    
    fig.add_trace(go.Scatter(x=fit_point, y=fit_point, mode='lines', 
                             line={'color': 'black'}, name='Perfect prediction'),
                   row=1, col=j)
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', 
                      showline=True, linewidth=2, linecolor='black', mirror=True,
                      title_text='True response', range=[min_lim_axis, max_lim_axis])
    
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', 
                      showline=True, linewidth=2, linecolor='black', mirror=True,
                      title_text='Predicted response', range=[min_lim_axis, max_lim_axis])
    if j > 1:
      fig.update_traces(row=1, col=j, showlegend=False)
    
    fig.layout.annotations[j-1].update(font=dict(size=20))

    j+=1

  fig.update_layout(
      template='simple_white',
      width=1780,
      height=650,
      title_text='<b>'+title_fig+'</b>',
      title_x=0.5, 
      font=dict(size=16), 
      legend_tracegroupgap=360,
      legend=dict(
          font=dict(size=16),
          bordercolor="Black",
          borderwidth=2
          )
  )

  fig.show()

### *Residual bar plot*
*Residual bar plot* will display blue points which represent the real observations, the yellow points which are predicted observations and the red bar which are the residual values between real and predicted values.

In [ ]:
def plot_residual_bar(df, output_names, title_fig):

  n_observations = df.shape[0]
  n_output = df.shape[1]
  
  fig = make_subplots(
      rows=1, cols= round(n_output/2),
      subplot_titles=['<b>' + name_output + '</b>' for name_output in output_names],
  )

  max_lim_axis = round(max(df.max(axis=0))+1)

  min_lim_axis = round(min(df.min(axis=0)))
  if min_lim_axis > 0:
    min_lim_axis = 0
  
  index_row = np.linspace(0, n_observations, n_observations)

  j = 1
  show_error_legend_bar = True

  for i in range(0,n_output,2):
    param_df = df.iloc[:, [i, i+1]]
    param_df = param_df.sort_values(by=param_df.columns[0])
    
    obs = param_df.iloc[:, 0].to_numpy()
    pred = param_df.iloc[:, 1].to_numpy()

    for k in range(1, n_observations):
      fig.add_trace(go.Scatter(x=[k,k], y=[obs[k], pred[k]], name="Error", 
                                mode='lines', line={'color': '#D62728', 'width':1},
                               showlegend=show_error_legend_bar), row=1, col=j)
      if show_error_legend_bar:
        show_error_legend_bar=False

    fig.add_trace(go.Scatter(x=index_row, y=obs, mode='markers', marker_color='#1F77B4', 
                             marker_size=8, name='True'), row=1, col=j)
    
    fig.add_trace(go.Scatter(x=index_row, y=pred, mode='markers', marker_color='#FF7F0E', 
                             marker_size=8, name='Predicted'), row=1, col=j)
        
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', 
                      showline=True, linewidth=2, linecolor='black', mirror=True,
                      title_text='Record number', range=[0, n_observations])
    
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', 
                      showline=True, linewidth=2, linecolor='black', mirror=True,
                      title_text='Response', range=[min_lim_axis, max_lim_axis])
    if j > 1:
      fig.update_traces(row=1, col=j, showlegend=False)
    
    fig.layout.annotations[j-1].update(font=dict(size=20))

    j+=1

  fig.update_layout(
      template='simple_white',
      width=1780,
      height=650,
      title_text='<b>'+title_fig+'</b>',
      title_x=0.5, 
      font=dict(size=16), 
      legend_tracegroupgap=360,
      legend=dict(
          font=dict(size=16),
          bordercolor="Black",
          borderwidth=2
          )
  )

  fig.show()

### *Compare observations plot*

In [ ]:
def plot_compare_observations(df, output_names, title_fig):

  n_observations = df.shape[0]
  n_output = df.shape[1]
  
  fig = make_subplots(
      rows=round(n_output/2), cols=1,
      subplot_titles=['<b>' + name_output + '</b>' for name_output in output_names],
  )

  max_lim_axis = round(max(df.max(axis=0))+1)

  min_lim_axis = round(min(df.min(axis=0)))
  if min_lim_axis > 0:
    min_lim_axis = 0
  
  index_row = np.linspace(0, n_observations, n_observations)

  j = 1

  for i in range(0,n_output,2):
    obs = df.iloc[:, i].to_numpy()
    pred = df.iloc[:, i+1].to_numpy()

    fig.add_trace(go.Scatter(x=index_row, y=obs, 
                             mode='lines', line={'color': '#1F77B4', 'width':1},
                             name='True'), row=j, col=1)
    
    fig.add_trace(go.Scatter(x=index_row, y=pred, 
                             mode='lines', line={'color': '#FF7F0E', 'width':1}, 
                             name='Predicted'),row=j, col=1)
    
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', 
                      showline=True, linewidth=2, linecolor='black', mirror=True,
                      title_text='Record number', range=[0, n_observations])
    
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', 
                      showline=True, linewidth=2, linecolor='black', mirror=True,
                      title_text='Response', range=[min_lim_axis, max_lim_axis])
    if j > 1:
      fig.update_traces(row=j, col=1, showlegend=False)
    
    fig.layout.annotations[j-1].update(font=dict(size=20))

    j+=1

  fig.update_layout(
      template='simple_white',
      width=1780,
      height=1500,
      title_text='<b>'+title_fig+'</b>',
      title_x=0.5, 
      font=dict(size=16), 
      legend_tracegroupgap=360,
      legend=dict(
          font=dict(size=16),
          bordercolor="Black",
          borderwidth=2
          )
  )

  fig.show()

## Plot regression results

In [ ]:
#summary_prediction[['b']] = summary_prediction.loc[:, ['a']]
#summary_prediction[['b_pred']] = summary_prediction.loc[:, ['a_pred']]
#summary_prediction[['c']] = summary_prediction.loc[:, ['a']]
#summary_prediction[['c_pred']] = summary_prediction.loc[:, ['a_pred']]
#summary_prediction

In [ ]:
plot_perfect_fit(summary_prediction,y_test.columns.tolist(), 'Cube')
#plot_perfect_fit(summary_prediction,['a','b','c'], 'Cube')

In [ ]:
plot_residual_bar(summary_prediction,y_test.columns.tolist(), 'Cube')
#plot_residual_bar(summary_prediction,['a','b','c'], 'Cube')

In [ ]:
plot_compare_observations(summary_prediction,y_test.columns.tolist(), 'Cube')